## Imports and Config

In [24]:
import ccxt

import numpy as np
import pandas as pd

from datetime import datetime, timedelta

from fredapi import Fred

## Load CSV File

In [ ]:
# dateparse = lambda dates: [datetime.strptime(d, '%Y-%m-%d %H:%M:%S') for d in dates]

In [124]:
df = pd.read_csv('IS5006_Historical_v2.csv', index_col='datetime', parse_dates=[0], dayfirst=True)
df.head()

,Fuzzy_sentiment_signal,MA_Signal
datetime,,
2021-01-01,0,0
2021-01-02,0,0
2021-01-03,0,0
2021-01-04,0,0
2021-01-05,0,0


## Get Historical Prices

In [26]:
exch = 'hitbtc'
period = '1d'
start = datetime(2020, 10, 31) # Earlier date to facilitate moving average calculation
end = datetime(2022, 3, 1)
limit = 1000
days = (end - start).days
symbol = 'BTC/USDT'

In [29]:
historical_df = pd.DataFrame(columns=['datetime', 'Open', 'High', 'Low', 'Close', 'Volume'])
exchange = getattr(ccxt, exch)()
exchange.load_markets()
for i in range(int(days//limit)+1):
  since = int((start+timedelta(limit*i)).timestamp())*1000
  history = exchange.fetch_ohlcv(symbol, period, since, limit)
  temp_df = pd.DataFrame(history)
  temp_df.columns = ['datetime', 'Open', 'High', 'Low', 'Close', 'Volume']
  temp_df['datetime'] = pd.to_datetime(temp_df['datetime'], unit='ms')
  historical_df = historical_df.append(temp_df)
historical_df.head()

,datetime,Open,High,Low,Close,Volume
0,2020-10-31,13555.19,14101.91,13420.00,13798.99,26926.29303
1,2020-11-01,13785.26,13889.95,13617.36,13757.07,21207.00752
2,2020-11-02,13758.98,13833.38,13205.48,13550.57,22176.92171
3,2020-11-03,13551.75,14069.41,13281.98,14021.61,19809.05179
4,2020-11-04,14023.11,14259.03,13527.50,14141.88,30407.10620


## Add MA Crossover

In [38]:
ma_df = historical_df.copy(deep = True)

In [39]:
ma_df['EMA_10'] = ma_df['Close'].ewm(span=10, adjust = False).mean()
ma_df['SMA_50'] = ma_df['Close'].rolling(50).mean()
ma_df[f'MA_Position'] = np.where(ma_df[f'EMA_10'] > ma_df[f'SMA_50'], 1.0, 0.0)
ma_df[f'MA_Signal'] = ma_df[f'MA_Position'].diff()
ma_df.drop([f'MA_Position'], axis=1, inplace=True)
ma_df.set_index('datetime', inplace=True)
ma_df.index = pd.to_datetime(ma_df.index, errors='coerce')
ma_df.head()

,Open,High,Low,Close,Volume,EMA_10,SMA_50,MA_Signal
datetime,,,,,,,,
2020-10-31,13555.19,14101.91,13420.00,13798.99,26926.29303,13798.990000,NaN,NaN
2020-11-01,13785.26,13889.95,13617.36,13757.07,21207.00752,13791.368182,NaN,0.0
2020-11-02,13758.98,13833.38,13205.48,13550.57,22176.92171,13747.586694,NaN,0.0
2020-11-03,13551.75,14069.41,13281.98,14021.61,19809.05179,13797.409113,NaN,0.0
2020-11-04,14023.11,14259.03,13527.50,14141.88,30407.10620,13860.040184,NaN,0.0


In [67]:
updated_df = df.merge(ma_df[['MA_Signal']], left_on=df.index, right_on=ma_df[['MA_Signal']].index, how='outer')
updated_df.rename({'key_0': 'datetime'}, inplace=True, axis=1)
updated_df.set_index('datetime', inplace=True)
updated_df.sort_index(ascending=True, inplace=True)
updated_df.head()

,fuzzy_2,fuzzy_2_signal,MA_Signal
datetime,,,
2020-10-31,NaN,NaN,NaN
2020-11-01,NaN,NaN,0.0
2020-11-02,NaN,NaN,0.0
2020-11-03,NaN,NaN,0.0
2020-11-04,NaN,NaN,0.0


In [68]:
updated_df.to_csv('IS5006_Historical.csv')

## Add Macroeconomic Data

In [72]:
macro_series = {'inflation': 'FPCPITOTLZGUSA', 'unemployment': 'UNRATE', 
        'oil': 'DCOILWTICO', 'gold_vol': 'GVZCLS'}

In [73]:
fred = Fred(api_key='40ed0248dfecd2195c2bc8353707eead')

In [89]:
macro_df = pd.DataFrame(index=df.index)
for key in macro_series.keys():
  temp_df = fred.get_series(macro_series[key])
  temp_df.name = key
  macro_df = macro_df.merge(temp_df, left_on=macro_df.index, right_on=temp_df.index, how='outer')
  macro_df.rename({'key_0': 'datetime'}, inplace=True, axis=1)
  macro_df.set_index('datetime', inplace=True)
  macro_df.sort_index(ascending=True, inplace=True)

In [103]:
macro_df.head()

,inflation,unemployment,oil,gold_vol
datetime,,,,
1948-01-01,NaN,3.4,NaN,NaN
1948-02-01,NaN,3.8,NaN,NaN
1948-03-01,NaN,4.0,NaN,NaN
1948-04-01,NaN,3.9,NaN,NaN
1948-05-01,NaN,3.5,NaN,NaN


In [105]:
macro_df['inflation'] = macro_df['inflation'].fillna(method="ffill")
macro_df['unemployment'] = macro_df['unemployment'].fillna(method="ffill")
macro_df['oil'] = macro_df['oil'].interpolate(method='time')
macro_df['gold_vol'] = macro_df['gold_vol'].interpolate(method='time')
macro_df.tail()

,inflation,unemployment,oil,gold_vol
datetime,,,,
2022-10-02,1.233584,3.8,103.22,22.23
2022-11-01,1.233584,3.8,103.22,22.23
2022-11-02,1.233584,3.8,103.22,22.23
2022-12-01,1.233584,3.8,103.22,22.23
2022-12-02,1.233584,3.8,103.22,22.23


In [126]:
updated_df = df.merge(macro_df, left_on=df.index, right_on=macro_df.index, how='left')
updated_df.rename({'key_0': 'datetime'}, inplace=True, axis=1)
updated_df.set_index('datetime', inplace=True)
# updated_df.sort_index(ascending=True, inplace=True)
updated_df.head()

,Fuzzy_sentiment_signal,MA_Signal,inflation,unemployment,oil,gold_vol
datetime,,,,,,
2021-01-01,0,0,1.233584,6.4,48.13,21.0575
2021-01-02,0,0,1.233584,6.4,47.91,21.3250
2021-01-03,0,0,1.233584,6.4,47.69,21.5925
2021-01-04,0,0,1.233584,6.4,47.47,21.8600
2021-01-05,0,0,1.233584,6.4,49.78,22.1400


In [130]:
updated_df.index = pd.to_datetime(updated_df.index)
updated_df.to_csv('IS5006_Historical.csv')